In [1]:
!git clone https://ChristineTharian@bitbucket.org/ChristineTharian/deeplearning.git

Cloning into 'deeplearning'...
remote: Counting objects: 10, done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 1), reused 0 (delta 0)
Unpacking objects: 100% (10/10), done.


In [3]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 86.3MB 44kB/s 
     |████████████████████████████████| 3.8MB 42.6MB/s 
     |████████████████████████████████| 450kB 53.4MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalle

In [4]:
!pip install --upgrade keras

     |████████████████████████████████| 378kB 10.1MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
!pip install h5py==2.7.0

     |████████████████████████████████| 4.8MB 6.5MB/s 
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0


In [0]:
!pip install keras==2.0.5

     |████████████████████████████████| 225kB 6.5MB/s 
  Created wheel for keras: filename=Keras-2.0.5-cp36-none-any.whl size=253391 sha256=84c4ca91b6609bee44dec293a62931710e79b5038577b732c9f35cc837d0d5cb
  Stored in directory: /root/.cache/pip/wheels/ee/09/df/2f10a3b36c903a620183396e24452e7de4dba01861b1d87cea
Successfully built keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.5 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
import tensorflow as tf
from tensorflow.keras.backend import set_session

sess = tf.Session()
set_session(sess)
sess.run(tf.global_variables_initializer())

In [0]:
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

Using TensorFlow backend.


In [0]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from random import shuffle
import os
import cv2

In [0]:
image_size = (64,64)

In [32]:
data = pd.read_csv('/content/deeplearning/fer2013.csv')
pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'), image_size)
    faces.append(face.astype('float32'))
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
emotions = pd.get_dummies(data['emotion']).as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [0]:
import sys, os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

In [0]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

x = faces
y = emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

# x -= np.mean(x, axis=0)
# x /= np.std(x, axis=0)

preprocess_input(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [0]:
from keras.models import load_model
from keras.models import model_from_json

#cnn = load_model('/content/fer.h5', compile=False)
# json_file = open('fer48.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# cnn = model_from_json(loaded_model_json)
# # load weights into new model
# cnn.load_weights("fer48.h5")

#simple = load_model('/content/cnnSimple.hdf5', compile=False)
xcept = load_model('/content/model_39.h5', compile=False)

In [0]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))
model.load_weights("fer48.h5")

In [0]:
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX, verbose = 1) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result

In [35]:
members = [xcept]
preds = ensemble_predictions(members, X_test)
temp = sum(np.argmax(y_test, axis=1) == preds)
temp/len(y_test) 

3589/3589 [==============================] - 23s 6ms/step


0.1306770688213987